In [4]:
import numpy as np
import itertools
import cv2
import random
import copy
from time import sleep, time
import pickle
import concurrent.futures
import os
import pandas as pd

In [1]:
import USDMM_v3 as USDMM

WOMC = USDMM.WOMC(
    new = True, # True/ STR -> If True inicialize with the cross operator/ If STR it will opens the file with the name passed
    nlayer = 2, # INT -> Number of operators in each layer
    wlen = 3,  # INT -> Size of the operator (wlen*wlen)
    train_size = 10, # INT -> Number of images to train 
    val_size = 10, # INT -> Number of images to validate
    test_size = 10, # INT -> Number of images to test
    error_type = 'iou', # 'mae' / 'iou' -> type of error
    neighbors_sample = False, # INT/False -> Number of neighbors to sort
    epoch_f = 10, # INT -> Number of epochs for the boolean function lattice (fixed windows)
    epoch_w = 50, # INT -> Number of epochs for the windows lattice
    batch = 10, # INT -> Batch size
    path_results = 'results_V2', # STR -> file where we want to save the results
    name_save='_V2', # STR -> pos fixed name for the results saved
    seed = 0, #INT -> seed for reproducibilit
    parallel = True, # True/False -> use parallel (True) or sequential (False)
    early_stop_round_f = 20, #INT -> max number of epochs without changes in the boolean function lattice
    early_stop_round_w = 10 #INT -> max number of epochs without changes in the windows lattice
)


Janela Inicializada: [array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan]), array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan])]
Resultados serão salvos em  results_V2
------------------------------------------------------------------


In [5]:
np.tanh(-100)

-1.0

In [6]:
par = WOMC.initialize_parameters_nn([np.array([np.nan,  1., np.nan,  1.,  1.,  1., np.nan,  1., np.nan]),np.array([np.nan,  1., np.nan,  1.,  1.,  1., np.nan,  1., np.nan])])
par

{'l0': {'W0': array([[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1,  1],
         [-1, -1, -1,  1, -1],
         [-1, -1, -1,  1,  1],
         [-1, -1,  1, -1, -1],
         [-1, -1,  1, -1,  1],
         [-1, -1,  1,  1, -1],
         [-1, -1,  1,  1,  1],
         [-1,  1, -1, -1, -1],
         [-1,  1, -1, -1,  1],
         [-1,  1, -1,  1, -1],
         [-1,  1, -1,  1,  1],
         [-1,  1,  1, -1, -1],
         [-1,  1,  1, -1,  1],
         [-1,  1,  1,  1, -1],
         [-1,  1,  1,  1,  1],
         [ 1, -1, -1, -1, -1],
         [ 1, -1, -1, -1,  1],
         [ 1, -1, -1,  1, -1],
         [ 1, -1, -1,  1,  1],
         [ 1, -1,  1, -1, -1],
         [ 1, -1,  1, -1,  1],
         [ 1, -1,  1,  1, -1],
         [ 1, -1,  1,  1,  1],
         [ 1,  1, -1, -1, -1],
         [ 1,  1, -1, -1,  1],
         [ 1,  1, -1,  1, -1],
         [ 1,  1, -1,  1,  1],
         [ 1,  1,  1, -1, -1],
         [ 1,  1,  1, -1,  1],
         [ 1,  1,  1,  1, -1],
         [ 1,  1,  1,  1,  

In [4]:
len(par)

2

In [6]:
par['l0']

{'W0': array([[-1, -1, -1, -1, -1],
        [-1, -1, -1, -1,  1],
        [-1, -1, -1,  1, -1],
        [-1, -1, -1,  1,  1],
        [-1, -1,  1, -1, -1],
        [-1, -1,  1, -1,  1],
        [-1, -1,  1,  1, -1],
        [-1, -1,  1,  1,  1],
        [-1,  1, -1, -1, -1],
        [-1,  1, -1, -1,  1],
        [-1,  1, -1,  1, -1],
        [-1,  1, -1,  1,  1],
        [-1,  1,  1, -1, -1],
        [-1,  1,  1, -1,  1],
        [-1,  1,  1,  1, -1],
        [-1,  1,  1,  1,  1],
        [ 1, -1, -1, -1, -1],
        [ 1, -1, -1, -1,  1],
        [ 1, -1, -1,  1, -1],
        [ 1, -1, -1,  1,  1],
        [ 1, -1,  1, -1, -1],
        [ 1, -1,  1, -1,  1],
        [ 1, -1,  1,  1, -1],
        [ 1, -1,  1,  1,  1],
        [ 1,  1, -1, -1, -1],
        [ 1,  1, -1, -1,  1],
        [ 1,  1, -1,  1, -1],
        [ 1,  1, -1,  1,  1],
        [ 1,  1,  1, -1, -1],
        [ 1,  1,  1, -1,  1],
        [ 1,  1,  1,  1, -1],
        [ 1,  1,  1,  1,  1]]),
 'b0': array([[-4],
        [-4]

In [9]:
for i in par:
    for j in par[i]:
        print(f'{i} - {j} - {par[i][j].shape}')

l0 - W0 - (32, 5)
l0 - b0 - (32, 1)
l0 - W1 - (1, 32)
l0 - b1 - (1, 1)
l1 - W0 - (32, 5)
l1 - b0 - (32, 1)
l1 - W1 - (1, 32)
l1 - b1 - (1, 1)


In [13]:
par['W2']

array([[ 0.0119345 ],
       [-0.19912914],
       [ 0.07604493],
       [-0.04787002],
       [-0.08279938]])

In [4]:
layer_dims = [5, 32, 1, 5, 32, 1]

In [9]:
np.zeros((1, 1))

array([[0.]])

In [4]:
np.array([-5+1]*1).reshape(1,1)

array([[-4]])

In [4]:
np.array([-5+1]*layer_dims[index+1]).reshape(layer_dims[index+1],1)

[5.0, 32.0, 1, 5.0, 32.0, 1]

In [6]:
l = [5.0, 32.0, 1, 5.0, 32.0, 1]
int(l)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [6]:
np.random.randn(par[1+1], 32.) * 0.1

TypeError: 'float' object cannot be interpreted as an integer